## 20 Exception Handling Practice (Basic → Advanced)

Run the next code cell. Each item prints a short result so you can see what happened.

Topics covered:
- `try/except`, multiple `except`, `else`, `finally`
- Raising exceptions (`raise`), re-raising, exception chaining (`raise ... from ...`)
- Custom exceptions, validation
- Reading files & parsing JSON safely
- Working with tracebacks
- Context managers (`with`), `contextlib.suppress`, `ExitStack`
- Retry with backoff (simple)
- Python 3.11+ `ExceptionGroup` / `except*` (optional)


In [2]:
#Handle ZeroDivisionError
def divide_numbers(num1, num2):
    try:
        result = num1 / num2
    except ZeroDivisionError:
        return "Error: Division by zero is not allowed."
    else:
        return result

In [3]:
#handle FileNotFoundError
def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
    except FileNotFoundError:
        return "Error: File not found."
    else:
        return content

In [4]:
#Multiple except blocks
def handle_exceptions(num1, num2, file_path):
    try:
        result = num1 / num2
        with open(file_path, 'r') as file:
            content = file.read()
    except ZeroDivisionError:
        return "Error: Division by zero is not allowed."
    except FileNotFoundError:
        return "Error: File not found."
    else:
        return result, content

In [5]:
#try–except–else block
def safe_divide(num1, num2):
    try:
        result = num1 / num2
    except ZeroDivisionError:
        return "Error: Division by zero is not allowed."
    else:
        return result

In [6]:
#try–finally block
def ensure_cleanup(num1, num2):
    try:
        result = num1 / num2
    except ZeroDivisionError:
        return "Error: Division by zero is not allowed."
    finally:
        print("Execution completed.")
    return result

In [7]:
#Handle ZeroDivisionError --- IGNORE ---
try:
    lst = [1, 2]
    print(lst[5])
except IndexError:
    print("Index out of range")


Index out of range


In [8]:
#Handle KeyError
try:
    d = {"a": 1}
    print(d["b"])
except KeyError:
    print("Key not found")


Key not found


### More Exception Handling Examples (8–20)

These continue from your earlier cells, moving into more advanced patterns.


In [ ]:
import json
import time
import traceback
from contextlib import suppress


def banner(title: str) -> None:
    print("\n" + "-" * 70)
    print(title)
    print("-" * 70)


# 8) ValueError / TypeError with multiple except
banner("8) Multiple except types (ValueError, TypeError)")
for value in ["123", "abc", None]:
    try:
        print(int(value))
    except (ValueError, TypeError) as exc:
        print("Handled", type(exc).__name__, "for", repr(value))


# 9) try/except/else (else only runs when no exception)
banner("9) try/except/else")
try:
    n = int("50")
except ValueError:
    print("Bad input")
else:
    print("Parsed successfully:", n)


# 10) finally (cleanup always runs)
banner("10) finally cleanup (manual close)")
f = None
try:
    f = open("demo_cleanup.txt", "w", encoding="utf-8")
    f.write("hello")
    print("Wrote demo_cleanup.txt")
except OSError as exc:
    print("File error:", exc)
finally:
    if f is not None:
        f.close()
        print("Closed file")


# 11) Raising your own exception (validation)
banner("11) raise ValueError for validation")
def set_percentage(p: int) -> int:
    if not (0 <= p <= 100):
        raise ValueError("percentage must be between 0 and 100")
    return p

try:
    set_percentage(250)
except ValueError as exc:
    print("Handled:", exc)


# 12) Custom exception
banner("12) Custom exception")
class ValidationError(Exception):
    pass


def validate_username(name: str) -> str:
    if not isinstance(name, str) or not name:
        raise ValidationError("username must be a non-empty string")
    if " " in name:
        raise ValidationError("username cannot contain spaces")
    return name

try:
    validate_username("bad name")
except ValidationError as exc:
    print("Handled ValidationError:", exc)


# 13) Exception chaining (raise ... from ...)
banner("13) Exception chaining (raise from)")
def load_json(text: str):
    try:
        return json.loads(text)
    except json.JSONDecodeError as exc:
        raise ValueError("Invalid JSON") from exc

try:
    load_json("{bad json}")
except ValueError as exc:
    print("Top-level:", exc)
    print("Cause:", type(exc.__cause__).__name__)


# 14) Re-raise after logging (keep original traceback)
banner("14) Re-raise after logging")
def parse_positive_int(text: str) -> int:
    try:
        value = int(text)
        if value <= 0:
            raise ValueError("must be > 0")
        return value
    except ValueError:
        print("parse_positive_int failed for", repr(text))
        raise

try:
    parse_positive_int("-5")
except ValueError as exc:
    print("Caller handled:", exc)


# 15) Getting traceback text (debugging)
banner("15) traceback.format_exc()")
try:
    int("not-a-number")
except Exception:
    print(traceback.format_exc().splitlines()[-1])


# 16) contextlib.suppress for expected errors
banner("16) suppress")
d = {"a": 1}
with suppress(KeyError):
    del d["missing"]
print("Still running; d=", d)


# 17) Retry with simple backoff
banner("17) Retry with backoff")
def retry(func, attempts: int = 3, delay_seconds: float = 0.2):
    for i in range(1, attempts + 1):
        try:
            return func()
        except Exception:
            if i == attempts:
                raise
            time.sleep(delay_seconds * i)

counter = {"n": 0}

def flaky():
    counter["n"] += 1
    if counter["n"] < 3:
        raise RuntimeError("temporary failure")
    return "success on attempt " + str(counter["n"])

print(retry(flaky, attempts=5))


# 18) Don't over-catch: Exception vs BaseException
banner("18) Exception vs BaseException")
try:
    raise SystemExit("exiting")
except Exception:
    print("This will NOT run")
except BaseException as exc:
    print("Caught BaseException:", type(exc).__name__)


# 19) Exception.add_note() (Python 3.11+)
banner("19) Exception.add_note() (Python 3.11+)")
try:
    raise ValueError("bad value")
except ValueError as exc:
    if hasattr(exc, "add_note"):
        exc.add_note("Tip: check the input string before converting")
    print("Handled:", exc)


# 20) ExceptionGroup / except* (Python 3.11+)
banner("20) ExceptionGroup / except* (Python 3.11+)")
try:
    ExceptionGroup
except NameError:
    print("ExceptionGroup not available on this Python version")
else:
    try:
        raise ExceptionGroup(
            "multiple errors",
            [ValueError("bad value"), TypeError("bad type"), ValueError("another")],
        )
    except* ValueError as eg:
        print("Handled ValueError group size:", len(eg.exceptions))
    except* TypeError as eg:
        print("Handled TypeError group size:", len(eg.exceptions))
